# load the data

In [18]:
from pprint import pprint
import json, os

# load a scrapped dataset
# json_path = r'C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\dFBApy\scraping\SABIO\scraped_content\sabio_proccessed.json' # this is for some reason not functional
json_path = 'sabio_proccessed.json'

with open(json_path) as data:
    sabio = json.load(data)
# sabio = json.load(open(json_path, 'r'))
# pprint(sabio, depth = 1)

# parse and consolidate the data

In [116]:
from scipy.constants import micro, nano, milli, minute
import re

def isnumber(num):
    try:
        float(num)
        return True
    except:
        try:
            int(num)
            return True
        except:
            return False
    
def average(num_1, num_2 = None):
    if isnumber(num_1): 
        if isnumber(num_2):
            numbers = [float(num_1), float(num_2)]
            average = sum(numbers) / len(numbers)
            return average
        else:
            return float(num_1)
    elif isnumber(num_2): 
        return float(num_2)
    elif type(num_1) is list:
        average = sum(num_1) / len(num_1)
        return average
    else:
        return None
    
def _determine_parameter_value(unit, original_value):
    # parse the unit
    numerator = ''
    denominator = ''
    term = ''
    skips = 0
    next_denominator = False
    for index in range(len(unit)):
        if skips > 0:
            skips -= 1 
            continue
            
        ch = unit[index]
        term += ch
        
        # parse the unit characters
        if len(unit) == 1:
            numerator += term
            term = ''
            break
        if index+1 == len(unit)-1:
            if next_denominator:
                denominator += term
                denominator += unit[index+1]
            else:
                numerator += term
                numerator += unit[index+1]
            term = ''
            break
        if unit[index+1] == '^':
            if unit[index+2:index+6] == '(-1)':
                denominator += term 
                skips = 5
                term = ''
            else:
                print(unit, term)
        elif unit[index+1] == '/':
            numerator += term
            term = ''
            skips = 1
            next_denominator = True
            
    if term != '':
        print(unit, term)
    unit_dic = {
        'numerator':numerator,
        'denominator': denominator
    }
    
    # determine the mathematically equivalent value in base units
    value = original_value
    for group in unit_dic:
        term = unit_dic[group]
        if re.search('min', unit_dic[group]):
            if group == 'numerator':
                value *= minute
                unit_dic[group] = re.sub('min', 's', unit_dic[group])
            else:
                value /= minute  
                unit_dic[group] = re.sub('min', 's', unit_dic[group])
        if re.search('mg|mM|mmol', unit_dic[group]):
            if group == 'numerator':
                value *= milli
                unit_dic[group] = re.sub('m', '', unit_dic[group], count = 1)
            else:
                value /= milli    
                unit_dic[group] = re.sub('m', '', unit_dic[group])
        if re.search('ng|nM|nmol', unit_dic[group]):
            if group == 'numerator':
                value *= nano
                unit_dic[group] = re.sub('n', '', unit_dic[group])
            else:
                value /= nano   
                unit_dic[group] = re.sub('n', '', unit_dic[group])
        if re.search('µ|u00b5|U\+00B5', unit_dic[group]):
            if group == 'numerator':
                value *= micro
                unit_dic[group] = re.sub('µ|u00b5g|U\+00B5', '', unit_dic[group])
            else:
                value /= micro 
                unit_dic[group] = re.sub('µ|u00b5g|U\+00B5', '', unit_dic[group])
                
        if term != unit_dic[group]:
            print(term, unit_dic[group])
    return value, unit_dic
            
# parse the values          
unit_prevalence = {}
consolidated_sabio = {}
missing_content = {}
pars = {}
blank_parameters = []
junk_entryids = []
examined_entry_ids = set()
repeated_entry_ids = []
for enzyme in sabio:
#     print('\n', enzyme, '\n', '='*2*len(enzyme))
    consolidated_sabio[enzyme] = {}
    for reaction in sabio[enzyme]:   
        consolidated_sabio[enzyme][reaction] = {}
        for entry_id in sabio[enzyme][reaction]:  
            if entry_id not in examined_entry_ids:
                examined_entry_ids.add(entry_id)
            else:
                repeated_entry_ids.append(entry_id)
            for par in sabio[enzyme][reaction][entry_id]['Parameters']:
                if par in pars:
                    pars[par] += 1
                elif par not in pars:
                    pars[par] = 1
                param = sabio[enzyme][reaction][entry_id]['Parameters'][par]
                if 'type' in param:
                    if 'unit' in param:
                        if not param['species'] == 'Enzyme':
                            initial_value = param['start val.']
                            final_value = param['end val.']
                            value = average(initial_value, final_value)
                            if value:
                                unit = param['unit']
                                if unit in unit_prevalence:
                                    unit_prevalence[unit] += 1
                                else:
                                    unit_prevalence[unit] = 1
                                
                                # standardize the units to base units
                                value, unit_dic = _determine_parameter_value(unit, value)
#                                 print(unit_dic)
                            else:
#                                 print('\n\n',par, )
#                                 pprint(param)
                                if param['type'] == 'concentration':
                                    blank_parameters.append(par)
                                else:
                                    junk_entryids.append(entry_id)
                elif 'unit' in param:
                    if param['unit'] == '°C':
                        temperature = param['start value']

print('parameters', )
pprint(pars)
print('\n\nunits', )
pprint(unit_prevalence)
print('\n\nunvalued concentration parameters')
pprint(blank_parameters)
print('\n\nentryids without essential parameters')
pprint(junk_entryids)
print('\n\nrepeated entryids')
pprint(repeated_entry_ids)
# print('\nmissing_content', )
# pprint(missing_content)

mM M
mM M
µM M
µM M
µM M
µM M
mg g
mM M
µM M
µM M
µM M
µM M
mg g
mM M
µM M
mM M
µM M
µM M
mM M
µM M
µM M
µM M
µM M
µM M
mM M
mM M
µM M
mM M
µM M
µM M
µM M
µM M
mg g
mM M
µM M
µM M
mg g
mM M
mM M
mM M
mM M
mM M
mM M
mM M
µM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
µM M
mM M
mM M
mM M
mM M
µmol mol
(min*mg) (s*g)
(min*g) (s*g)
(min*g) (s*g)
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
µM M
µmol mol
(min*mg) (s*g)
mM M
mM M
µM M
mM M
µM M
µmol mol
(min*mg) (s*g)
µM M
mM M
mM M
mM M
µM M
µmol mol
(min*mg) (s*g)
mM M
mM M
mM M
µM M
µM M
µmol mol
(min*mg) (s*g)
mM M
mM*s M*s
mM M
mM*s M*s
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
mM M
µmol mol
(min*mg) (s*g)
mM M
mM M
min s
µM*min M*s
µM 